# Ejercicio:
Dada la data de geyser.csv, utilizar el agloritmo Gaussiano para entrenar en la detección de anomalías. Intentar probar con diferentes percentiles.

El conjunto de datos contiene información sobre las erupciones del géiser Old Faithful localizado en el parque nacional de Yellowstone, Wyoming. En concreto, recoge información sobre la duración de 299 erupciones, así como el tiempo transcurrido desde la anterior

## Lectura conjunto de datos y EDA

In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec
from collections import Counter
from sklearn import metrics
import numpy as np
from matplotlib.colors import LogNorm
from sklearn.metrics import f1_score
from sklearn.mixture import GaussianMixture

In [3]:
df_geyser = pd.read_csv('geyser.csv')
df_geyser

,duration,waiting,kind
0,3.600,79,long
1,1.800,54,short
2,3.333,74,long
3,2.283,62,short
4,4.533,85,long
...,...,...,...
267,4.117,81,long
268,2.150,46,short
269,4.417,90,long
270,1.817,46,short


In [4]:
# Comprobamos si alguna columna tiene valores nulos
df_geyser.isna().any()

duration    False
waiting     False
kind        False
dtype: bool

## Detección de anomalías con Distribución Gaussiana

In [5]:
# Se reduce el DataFrame eliminando la columna 'kind'
df_geyser.drop(columns='kind', inplace=True)
df_geyser.head()

,duration,waiting
0,3.600,79
1,1.800,54
2,3.333,74
3,2.283,62
4,4.533,85


### Entrenamiento del algoritmo

In [7]:
# Se entrena el algoritmo Gaussiano
gm = GaussianMixture(n_components=2, random_state=42)
gm.fit(df_geyser)

GaussianMixture(n_components=2, random_state=42)

### Representación del límite de decisión

In [ ]:
def plot_gaussian_mixture(clusterer, X, y, resolution=1000):
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    Z = -clusterer.score_samples(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z,
                 norm=LogNorm(vmin=1.0, vmax=30.0),
                 levels=np.logspace(0, 2, 12))
    plt.contour(xx, yy, Z,
                norm=LogNorm(vmin=1.0, vmax=30.0),
                levels=np.logspace(0, 2, 12),
                linewidths=1, colors='k')

    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], 'k.', markersize=2)
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], 'r.', markersize=2)

### Anomalías identificadas

Para la identificación de las anomalías, se debe seleccionar un threshold a partir del cual, todos los ejemplos que se encuentren en regiones con una densidad menor a la indicada en el threshold, se consideran anomalías.

#### Percentile 0.03

In [19]:
# Selección del Threshold
densities = gm.score_samples(df_geyser)
density_threshold = np.percentile(densities, 0.03)
print("Threshold seleccionado:", density_threshold)

Threshold seleccionado: -8.77247217446857


In [20]:
# Se agrega una columna llamada 'anomaly' al DataFrame
df_geyser['anomaly'] = (densities < density_threshold).astype(int)
df_geyser['anomaly'].value_counts()

,duration,waiting,anomaly
0,3.600,79,0
1,1.800,54,0
2,3.333,74,0
3,2.283,62,0
4,4.533,85,0
...,...,...,...
267,4.117,81,0
268,2.150,46,0
269,4.417,90,0
270,1.817,46,0


#### Percentile 0.1

In [22]:
# Selección del Threshold
df_geyser.drop(columns='anomaly', inplace=True)
densities = gm.score_samples(df_geyser)
density_threshold = np.percentile(densities, 0.1)
print("Threshold seleccionado:", density_threshold)

Threshold seleccionado: -8.73016713908708


In [23]:
# Se agrega una columna llamada 'anomaly' al DataFrame
df_geyser['anomaly'] = (densities < density_threshold).astype(int)
df_geyser['anomaly'].value_counts()

anomaly
0    271
1      1
Name: count, dtype: int64

### Búsqueda del mejor Threshold

In [26]:
X = df_geyser.drop(columns='anomaly')
y = df_geyser['anomaly']

In [24]:
from sklearn.base import BaseEstimator

class GaussianAnomalyDetector(BaseEstimator):
    def __init__(self, threshold=1):
        self._threshold = threshold
        self._gm = None
    def fit(self, X, y=None):
        self._gm = GaussianMixture(n_components=2, n_init=10, random_state=42)
        self._gm.fit(X) 
        return self
    
    def predict(self, X, y=None):
        densities = self._gm.score_samples(X)
        y_preds = (densities < self._threshold)
        y_preds[y_preds == False] = 0
        y_preds[y_preds == True] = 1
        return y_preds
    
    def get_params(self, deep=True):
        return {"threshold": self._threshold}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
            return self

In [27]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

gad = GaussianAnomalyDetector()

param_distribs = {
    # Utiliza 'uniform' para distribuciones continuas. 
    # 'loc' es el inicio del rango y 'scale' es la anchura del rango (el total de valores en el rango).
    'threshold': uniform(loc=0.01, scale=4.99),
}

# Configuración de RandomizedSearchCV (5*3=15 rondas de entrenamiento)
rnd_search = RandomizedSearchCV(gad, param_distributions=param_distribs,
                                n_iter=5, cv=3, scoring='f1')

# Entrenamiento del modelo
rnd_search.fit(X, y)

RandomizedSearchCV(cv=3, estimator=GaussianAnomalyDetector(), n_iter=5,
                   param_distributions={'threshold': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x29967fdf0>},
                   scoring='f1')

In [28]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.007246376811594204 {'threshold': 4.149142389759702}
0.007246376811594204 {'threshold': 0.3540911423966426}
0.007246376811594204 {'threshold': 2.306323234824726}
0.007246376811594204 {'threshold': 1.0698979579586199}
0.007246376811594204 {'threshold': 4.386762274174451}


La forma mostrada anteriormente es la más estándar a la hora de realizar la búsqueda, pero también es la más ineficiente. Con el método anterior se requiere entrenar el modelo 15 veces (con los parámetro indicados). 

Con la forma que se muestra a continuación solo es necesario entrenar el modelo una única vez.

In [29]:
from sklearn.metrics import precision_score

def select_threshold(list_thds, densities, y):
    best_prec = 0
    best_threshold = 0
    i = 0
    for thd in list_thds:
        i += 1
        print("\rSearching best threshold {0}%".format(
            int((i + 1) / len(list_thds) * 100)), end='')
        preds = (densities < thd)
        preds[preds == False] = 0
        preds[preds == True] = 1
        precision = precision_score(y, preds)
        if precision > best_prec:
            best_prec = precision
            best_threshold = thd
    return (best_prec, best_threshold)

In [30]:
select_threshold(np.arange(-600, -300, 1), densities, y)

Searching best threshold 87%

/Users/n.c.rodriguez/anaconda3/envs/nico/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/n.c.rodriguez/anaconda3/envs/nico/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/n.c.rodriguez/anaconda3/envs/nico/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/n.c.rodriguez/anaconda3/envs/n

Searching best threshold 100%

/Users/n.c.rodriguez/anaconda3/envs/nico/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/n.c.rodriguez/anaconda3/envs/nico/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/n.c.rodriguez/anaconda3/envs/nico/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/n.c.rodriguez/anaconda3/envs/n

(0, 0)